## Práctica 4

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score




### Apartado A

#### En primer lugar, haremos el preprocesamiento de datos para poder entrenar el clasificador.

#### A continuación, vamos a crear nuevas columnas para evaluar si la temperatura es alta, media o baja.
#### Además, crearemos otra columna que determine si el nivel de precipitación es alto, medio o bajo.

In [2]:

# Carga de los datos
data = pd.read_csv('./RecursosLab3/lluvias.csv')
data = data.dropna()
data = data.drop_duplicates()

# Eliminación de la columna "DATE"
data.drop("DATE", axis=1, inplace=True)

# Creación de la variable objetivo binaria
data["RAIN"] = data["RAIN"].astype(int)

# Creación de la columna de temperatura media diaria promedio
data["TEMP_MED"] = (data["TMAX"] + data["TMIN"]) / 2

# Categorización de las variables continuas
data["PRCP_CAT"] = pd.cut(data["PRCP"], bins=[-1, 1, 2, float("inf")], labels=["baja", "normal", "alta"])
data["TEMP_CAT"] = pd.cut(data["TEMP_MED"], bins=[-float("inf"), 10, 20, float("inf")], labels=["baja", "media", "alta"])

data



,PRCP,TMAX,TMIN,RAIN,TEMP_MED,PRCP_CAT,TEMP_CAT
0,0.47,51,42,1,46.5,baja,alta
1,0.59,45,36,1,40.5,baja,alta
2,0.42,45,35,1,40.0,baja,alta
3,0.31,45,34,1,39.5,baja,alta
4,0.17,45,32,1,38.5,baja,alta
...,...,...,...,...,...,...,...
25546,0.00,49,34,0,41.5,baja,alta
25547,0.00,49,29,0,39.0,baja,alta
25548,0.00,46,32,0,39.0,baja,alta
25549,0.00,48,34,0,41.0,baja,alta


#### Se considera que la temperatura es baja si está por debajo de 10 grados, media si está entre 10 y 20 y alta si está por encima de 20. 
#### Se considera que ha llovido poco si la medida está por debajo de 1, normal si está entre 1 y 2, y que ha llovido mucho si está por encima de 2.



### Apartado B

#### A continuación, crearemos un clasificador e indicaremos su error. Además, lo usaremos para saber si lloverá en los próximos 3 días.

#### Para clasificar usaremos el clasificador: arboles de decisión.

In [3]:

# Codificar variables categóricas usando one-hot encoding
# Esta técnica crea una nueva columna para cada valor categórico en una variable, y asigna un valor de 0 o 1 a cada fila según el valor correspondiente.
data_encoded = pd.get_dummies(data, columns=['TMIN', 'TMAX', 'PRCP', 'PRCP_CAT', 'TEMP_CAT'])

# Dividir datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(data_encoded.drop('RAIN',axis=1), data_encoded['RAIN'], test_size=0.3, random_state=101)

# Entrenar modelo
dtree = DecisionTreeClassifier()
dtree.fit(X_train,y_train)

# Evaluar modelo
predictions = dtree.predict(X_test)
print(accuracy_score(y_test,predictions))


1.0


##### Un resultado de 1.0 en la precisión del modelo significa que el modelo de árbol de decisión ha clasificado correctamente todas las muestras de prueba en el conjunto de datos utilizado.

##### En otras palabras, el modelo ha logrado una precisión del 100%, lo que indica que es capaz de predecir con precisión si va a llover o no en los próximos 3 días basado en las variables de temperatura y precipitación.

In [4]:

# Calcular la precisión del modelo
accuracy = accuracy_score(y_test, predictions)
print("Precisión del modelo:", accuracy)

# Calcular el error del modelo
error = 1 - accuracy
print("Error del modelo:", error)



Precisión del modelo: 1.0
Error del modelo: 0.0


##### El error del modelo es 0.0. Lo que indica que es capaz de predecir con precisión si va a llover o no en los próximos 3 días basado en las variables de temperatura y precipitación.

#### Una vez hemos creado el clasificador e indicado su error, vamos a usarlo para predecir si lloverá en los proximos 3 días o no.

In [5]:
import pandas as pd

# Codificar la temperatura y la cantidad de lluvia en formato One-Hot Encoding
temp = 'TMIN_BAJA'
prcp = 'PRCP_ALTO'
data = {'TMIN':[temp],'TMAX':[temp],'PRCP':[prcp],'PRCP_CAT':[prcp],'TEMP_CAT':[temp]}
data_encoded = pd.get_dummies(pd.DataFrame(data))

# Asegurarse de que los datos de predicción tengan las mismas columnas que los datos de entrenamiento
data_encoded = data_encoded.reindex(columns=X_train.columns, fill_value=0)

# Realizar la predicción
prediction = dtree.predict(data_encoded)
print(prediction)


[1]


#### Si el resultado es [1], se predice que lloverá. Si es 0, se predice que no lloverá. En este caso devuelve [1]. Por tanto, para los valores introducidos de "temp" y "prcp" ll

#### A continuación, usando una fuente externa, vamos a predecir si lloverá o no en los próximos 3 días

In [6]:
import pyowm
from datetime import datetime, timedelta

owm = pyowm.OWM(api_key='d4153e98656d58a73db9668369a26209')

# Definir la ubicación de la ciudad y la fecha de inicio y fin de la búsqueda del pronóstico del tiempo
city = "Málaga,ES"
start_date = datetime.now()
end_date = start_date + timedelta(days=3)

# Obtener el pronóstico del tiempo para la ubicación y el período de tiempo especificado
forecast = owm.weather_manager().forecast_at_place(city, interval='3h')

# Verificar si se espera lluvia en los próximos 3 días
for weather in forecast.forecast:
    if 'rain' in weather.status.lower():
        print(f"Se espera lluvia en {city} el {weather.reference_time('iso')} (UTC)")



#### El código devuelve el pronóstico del tiempo en intervalos de 3 horas para los próximos 3 días en Málaga y verifica si se espera lluvia en alguno de esos momentos.
#### Al no devolver nada, podemos concluir que no lloverá en Málaga en los próximos 3 días.